# RAG BASIC

In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# .env 파일 로드
load_dotenv()

# langSmith에 로깅 할 프로젝트 명을 입력
logging.langsmith("02-RAG BASIC")

## RAG 기본 파이프라인 (1~8단계)

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

아래는 기본적인 RAG 구조 이해를 위한 뼈대코드(skeleton code) 입니다.

각 단계별 모듈의 내용을 앞으로 상황에 맞게 변경하면서 문서에 적합한 구조를 찾아갈 수 있습니다.

(각 단계별로 다양한 옵션을 설정하거나 새로운 기법을 적용할 수 있습니다.)

In [9]:
# 1단계 : 문서 로드 (Load document)
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
print(len(docs))

23


1단계로 문서를 로드 하는 작업을 진행한다
PyMuPDFLoader 라이브러리를 이용하여 PDF 파일 문서를 불러올 수 있다. PDF를 불러오기 위한 다른 라이브러리도 있으며 다른 유형의 파일을 불러오기 위해 적절한 라이브러리를 찾아 로드하면 된다

PyMuPDFLoader를 이용하여 PDF 파일을 불러오면 페이지별로 데이터가 분류되는 것을 확인 할 수 있다
len 함수를 호출하여 docs의 페이지 길이가 원본 PDF 파일의 페이지 수와 동일한 것을 확인 할 수 있다

docs 객체는 page_content와 metadata 속성으로 나눠진다
page_content에는 페이지 정보 데이터를 확인 할 수 있고
metadata에는 페이지의 정보가 담겨져 있다

페이지의 내용을 출력한다

In [11]:
print(docs[1].page_content)

2023년 12월호
Ⅰ. 인공지능 산업 동향 브리프
 1. 정책/법제 
   ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표  ························· 1
   ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2
   ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3
   ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4
   ▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5
   ▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··················································· 6
 
 2. 기업/산업 
   ▹ 미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성································ 7
   ▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개  ······································· 8
   ▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9
   ▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10
   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11
   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달

`metadata`를 확인한다

In [13]:
docs[1].metadata

{'producer': 'Hancom PDF 1.3.0.542',
 'creator': 'Hwp 2018 10.0.0.13462',
 'creationdate': '2023-12-08T13:28:38+09:00',
 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'moddate': '2023-12-08T13:28:38+09:00',
 'trapped': '',
 'page': 1}

In [18]:
# 2단계 : 문서 분할 (Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

2단계로 문서를 분할 하는 작업을 진행한다
1단계에서 불러온 문서를 chunk 단위로 쪼개는 작업을 진행한다. chunk를 쪼개는 알고리즘에는 여러가지가 있으며 여기서는 그 수많은 것들 중 RecursiveCharacterTextSplitter 를 사용한다

RecursiveCharacterTextSplitter 는 요청한 chunk의 길이(size)만큼 짜른다. 길이만큼 짤랐을때 문단의 단절감을 줄이기 위해 overlap 값을 이용하여 chunk를 자를때 겹쳐서 자른다

In [21]:
# 분할된 청크수를 조회해 본다
f"분할된 청크의 수 : {len(split_documents)}"

'분할된 청크의 수 : 55'

In [47]:
# 청크 된 내용을 확인 해본다.
# 실제 페이지의 데이터와 비교해보는 것도 좋다
print(split_documents[29].page_content)

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성


In [50]:
# metadata 를 확인해 본다
split_documents[29].metadata

{'producer': 'Hancom PDF 1.3.0.542',
 'creator': 'Hwp 2018 10.0.0.13462',
 'creationdate': '2023-12-08T13:28:38+09:00',
 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'moddate': '2023-12-08T13:28:38+09:00',
 'trapped': '',
 'page': 12}

In [33]:
# 3단계 : 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x15f31a210>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x15f341810>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

3단계로 임베딩 작업을 진행한다

임베딩은 청크로 나눈 데이터들을 인덱싱 하는 작업으로 여기서는 OpenAI 의 임베딩을 사용하며 임베딩을 사용하게 되면 OpenAI Platform의 비용이 차감된다. 임베딩 역시 다양한 솔루션들이 있으니 추후 활용해 보는 것도 좋다

In [34]:
# 4단계 : DB 생성 및 저장
# 벡터스토어를 생성
vectorStore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

4단계로 벡터스토어를 생성하고 임베딩된 데이터를 저장한다

여기서는 FAISS라는 벡터스토어를 생성하고 데이터를 넣을 때 변수로 청크된 데이터와 임베딩 객체를 전달한다.
이때 임베딩이 실제로 진행되고 임베딩 된 결과가 벡터스토어에 저장된다

현재는 메모리상에 벡터 데이터가 저장된것으로 추후 과정을 통해 로컬에 저장하는 방법에 대해서도 확인 할 수 있다고 한다

In [42]:
# 벡터스토어에 저장 된 임베딩 데이터에서 유사도 검색을 하여 해당 청크 데이터를 출력
for doc in vectorStore.similarity_search("삼성"):
    print(doc)
    print("------------------------------")

page_content='SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성' metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subj

In [51]:
# 5단계 : 검색기 (Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성
retriever = vectorStore.as_retriever()

In [ ]:
# 검색기에 쿼리를 날려 검색된 Chunk 결과를 확인
retriever.invoke("삼성전자가 자체 개발한 AI의 이름은?")

In [55]:
# 6단계 : 프롬프트 생성 (Create prompt)
# 프롬프트를 생성한다

prompt = PromptTemplate.from_template(
    """
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in Korean.

    #Context:
    {context}

    #Question:
    {question}

    #Answer:
    """
)

6단계에서는 프롬프트를 생성한다. 
프롬프트에 들어가는 두 개의 변수는 context 와 question으로 context는 검색기에서 조회 된 결과가 들어가고 question에는 사용자의 질의가 들어간다

프롬프트에는 질의 업무를 수행하기 위한 어시스턴트임을 주입. 검색기에서 조회 된 context 기반으로 대답을 할것으로 요청하였으며 모르는 내용에 대해서는 모른다라고 대답 하길 요청하게 프롬프트를 작성하였다

In [56]:
# 7단계 : 언어모델 (LLM) 생성
# 모델을 생성한다
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.1)

In [57]:
# 8단계 : 체인 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm 
    | StrOutputParser()
)

In [61]:
# 체인 실행
# 문서에 대해 질의를 입력하고, 답변을 출력한다
question = "가우스의 특징은 뭐야?" 
response = chain.invoke(question)

print(response)

삼성 가우스의 특징은 다음과 같습니다:

1. **온디바이스 작동**: 삼성 가우스는 온디바이스에서 작동 가능하여 외부로 사용자 정보가 유출될 위험이 없습니다.

2. **다양한 모델 구성**: 언어, 코드, 이미지의 3개 모델로 구성되어 있습니다.
   - **언어 모델**: 메일 작성, 문서 요약, 번역 업무를 지원합니다.
   - **코드 모델**: AI 코딩 어시스턴트 '코드아이(code.i)'를 통해 대화형 인터페이스로 서비스를 제공하며, 사내 소프트웨어 개발에 최적화되어 있습니다.
   - **이미지 모델**: 창의적인 이미지 생성 및 기존 이미지 변환, 저해상도 이미지의 고해상도 전환을 지원합니다.

3. **안전한 데이터 학습**: 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었습니다.

4. **다양한 제품에 단계적 탑재 계획**: 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획입니다.
